In [1]:
import nltk
import random
from nltk.classify.scikitlearn import SklearnClassifier
import pickle
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.linear_model import LogisticRegression, SGDClassifier
from sklearn.svm import SVC
from nltk import pos_tag
from nltk import sent_tokenize, word_tokenize
from nltk.classify import ClassifierI
from statistics import mode
from nltk.tokenize import word_tokenize
import re
import bz2
import os
from nltk.stem import WordNetLemmatizer
from string import punctuation
from nltk.corpus import stopwords
from nltk.classify import SklearnClassifier


In [2]:
files_pos = os.listdir('/Projects/Jupyter notebook/')
print(files_pos)
train_file = bz2.BZ2File('/Projects/Jupyter notebook/train.ft.txt.bz2')
test_file=bz2.BZ2File('/Projects/Jupyter notebook/test.ft.txt.bz2')



['.ipynb_checkpoints', 'DE.ipynb', 'test.ft.txt.bz2', 'train.ft.txt.bz2']


In [3]:
train_file_lines=train_file.readlines()


In [4]:
train_file_lines = [x.decode('utf-8') for x in train_file_lines]

In [5]:
train_labels = [0 if x.split(' ')[0] == '__label__1' else 1 for x in train_file_lines]

In [6]:
train_sentences = [x.split(' ', 1)[1][:-1] for x in train_file_lines]


### Preprocessing

In [7]:
all_words = []
documents = []

import re
i=0
stop_words = list(set(stopwords.words('english')))

#  j is adject, r is adverb, and v is verb
#allowed_word_types = ["J","R","V"]
allowed_word_types = ["J"]
for i in range(0,len(train_labels)):
    documents.append((train_sentences[i],train_labels[i]))
len(documents)


3600000

In [8]:
for p in documents[0:30000]:
    # remove punctuations
    cleaned = re.sub(r'[^(a-zA-Z)\s]','', p[0])
    
    # tokenize 
    tokenized = word_tokenize(cleaned)
    
    # remove stopwords 
    stopped = [w for w in tokenized if not w in stop_words]
    
    # parts of speech tagging for each word 
    pos = nltk.pos_tag(stopped)
    
    # make a list of  all adjectives identified by the allowed word types list above
    for w in pos:
        if w[1][0] in allowed_word_types:
            all_words.append(w[0].lower())
                

In [9]:
save_documents = open("pickled_algos/documents.sav","wb")
pickle.dump(documents, save_documents)
save_documents.close()

In [10]:
len(all_words)

204954

In [11]:
len(pos)

20

In [12]:
# creating a frequency distribution of each adjectives. 
BOW = nltk.FreqDist(all_words)
BOW

FreqDist({'good': 8025, 'great': 6906, 'best': 2917, 'many': 2818, 'much': 2738, 'bad': 2464, 'little': 2464, 'first': 2359, 'new': 2264, 'old': 1963, ...})

In [13]:
# listing the 5000 most frequent words
word_features = list(BOW.keys())[:5000]
word_features[0], word_features[-1]

('beautiful', 'litany')

In [14]:
save_word_features = open("pickled_algos/word_features5k.sav","wb")
pickle.dump(word_features, save_word_features)
save_word_features.close()


In [15]:
# function to create a dictionary of features for each review in the list document.
# The keys are the words in word_features 
# The values of each key are either true or false for wether that feature appears in the review or not
def find_features(document):
    words = word_tokenize(document)
    features = {}
    for w in word_features:
        features[w] = (w in words)

    return features

# Creating features for each review
featuresets = [(find_features(rev), category) for (rev, category) in documents[0:30000]]
#featuresets_test=[(find_features(rev), category) for (rev, category) in document_test[0:20000]]
# Shuffling the documents 
random.shuffle(featuresets)
print(len(featuresets))

30000


In [16]:
training_set = featuresets[0:25000]
testing_set = featuresets[25000:]
print( 'training_set :', len(training_set), '\ntesting_set :', len(testing_set))

training_set : 25000 
testing_set : 5000


In [17]:
classifier = nltk.NaiveBayesClassifier.train(training_set)

print("Classifier accuracy percent:",(nltk.classify.accuracy(classifier, testing_set))*100)

classifier.show_most_informative_features(15)

Classifier accuracy percent: 81.14
Most Informative Features
                  refund = True                0 : 1      =    102.9 : 1.0
                   waste = True                0 : 1      =     31.2 : 1.0
                pathetic = True                0 : 1      =     28.2 : 1.0
                 rubbish = True                0 : 1      =     23.7 : 1.0
             embarrassed = True                0 : 1      =     21.6 : 1.0
               defective = True                0 : 1      =     21.4 : 1.0
             pretentious = True                0 : 1      =     20.9 : 1.0
                  poorly = True                0 : 1      =     19.9 : 1.0
                rambling = True                0 : 1      =     19.4 : 1.0
                   worst = True                0 : 1      =     18.2 : 1.0
                travesty = True                0 : 1      =     18.0 : 1.0
               worthless = True                0 : 1      =     17.8 : 1.0
                   awful = True        

In [18]:
# Printing the most important features 

mif = classifier.most_informative_features()

mif = [a for a,b in mif]
print(mif)

['refund', 'waste', 'pathetic', 'rubbish', 'embarrassed', 'defective', 'pretentious', 'poorly', 'rambling', 'worst', 'travesty', 'worthless', 'awful', 'publish', 'shoddy', 'letdown', 'timeless', 'tripe', 'unfunny', 'unusable', 'pointless', 'atrocious', 'disappointment', 'useless', 'forgettable', 'cancel', 'clumsy', 'horrendous', 'idiotic', 'warped', 'lousy', 'wasted', 'offensive', 'formatted', 'dreadful', 'disappointing', 'laughable', 'unreadable', 'trash', 'irrelevant', 'superficial', 'applicable', 'disservice', 'formulaic', 'jammed', 'nonsensical', 'horrible', 'shines', 'uninteresting', 'garbage', 'poor', 'worse', 'skeptical', 'drunk', 'underrated', 'coolest', 'rural', 'warranty', 'advertisement', 'ridiculous', 'delightful', 'cheated', 'overrated', 'lame', 'misleading', 'incomprehensible', 'terrible', 'abysmal', 'fluke', 'suction', 'trigger', 'hassle', 'incorrect', 'adventurous', 'gem', 'contrived', 'inferior', 'launch', 'talentless', 'watered', 'redeem', 'stink', 'downside', 'energe

In [19]:
# getting predictions for the testing set by looping over each reviews featureset tuple
# The first elemnt of the tuple is the feature set and the second element is the label 
ground_truth = [r[1] for r in testing_set]

preds = [classifier.classify(r[0]) for r in testing_set]


In [20]:
from sklearn.metrics import f1_score
f1_score(ground_truth, preds, labels = [0, 1], average = 'micro')

0.8114

In [21]:
from nltk.classify.scikitlearn import SklearnClassifier
from sklearn.naive_bayes import MultinomialNB,BernoulliNB
from sklearn.linear_model import LogisticRegression,SGDClassifier
from sklearn.svm import SVC

Classifiers for an ensemble model: 
Naive Bayes (NB)
Multinomial NB
Bernoulli NB
Logistic Regression
Stochastic Gradient Descent Classifier - SGD
Support Vector Classification - SVC


In [ ]:
#print("Original Naive Bayes Algo accuracy percent:", (nltk.classify.accuracy(classifier, testing_set))*100)
#classifier.show_most_informative_features(15)

MNB_clf = SklearnClassifier(MultinomialNB())
MNB_clf.train(training_set)
print("MNB_classifier accuracy percent:", (nltk.classify.accuracy(MNB_clf, testing_set))*100)

BNB_clf = SklearnClassifier(BernoulliNB())
BNB_clf.train(training_set)
print("BernoulliNB_classifier accuracy percent:", (nltk.classify.accuracy(BNB_clf, testing_set))*100)

LogReg_clf = SklearnClassifier(LogisticRegression())
LogReg_clf.train(training_set)
print("LogisticRegression_classifier accuracy percent:", (nltk.classify.accuracy(LogReg_clf, testing_set))*100)

SGD_clf = SklearnClassifier(SGDClassifier())
SGD_clf.train(training_set)
print("SGDClassifier_classifier accuracy percent:", (nltk.classify.accuracy(SGD_clf, testing_set))*100)

MNB_classifier accuracy percent: 81.08


In [ ]:
len(training_set)

In [ ]:
SVC_clf = SklearnClassifier(SVC()
SVC_clf.train(training_set)
print("SVC_classifier accuracy percent:", (nltk.classify.accuracy(SVC_clf, testing_set))*100)

### Storing all models using pickle 

In [ ]:
def create_pickle(c, file_name): 
    save_classifier = open(file_name, 'wb')
    pickle.dump(c, save_classifier)
    save_classifier.close()

classifiers_dict = {'ONB': [classifier, 'pickled_algos/ONB_clf.sav'],
                    'MNB': [MNB_clf, 'pickled_algos/MNB_clf.sav'],
                    'BNB': [BNB_clf, 'pickled_algos/BNB_clf.sav'],
                    'LogReg': [LogReg_clf, 'pickled_algos/LogReg_clf.sav'],
                    'SGD': [SGD_clf, 'pickled_algos/SGD_clf.sav']} 
                    #'SVC': [SVC_clf, 'pickled_algos/SVC_clf.pickle']}




for clf, listy in classifiers_dict.items(): 
    create_pickle(listy[0], listy[1])

In [ ]:
from sklearn.metrics import accuracy_score
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils.multiclass import unique_labels

In [ ]:
'''acc_scores = {}
for clf, listy in classifiers_dict.items(): 
    # getting predictions for the testing set by looping over each reviews featureset tuple
    # The first elemnt of the tuple is the feature set and the second element is the label 
    acc_scores[clf] = accuracy_score(ground_truth, predictions[clf])
    print(f'Accuracy_score {clf}: {acc_scores[clf]}')'''

In [ ]:
from sklearn.metrics import f1_score, accuracy_score
ground_truth = [r[1] for r in testing_set]
predictions = {}
f1_scores = {}
for clf, listy in classifiers_dict.items(): 
    # getting predictions for the testing set by looping over each reviews featureset tuple
    # The first elemnt of the tuple is the feature set and the second element is the label 
    predictions[clf] = [listy[0].classify(r[0]) for r in testing_set]
    f1_scores[clf] = f1_score(ground_truth, predictions[clf], labels = [0, 1], average = 'micro')
    print(f'f1_score {clf}: {f1_scores[clf]}')

# Ensemble Model

In [ ]:
from nltk.classify import ClassifierI

# Defininig the ensemble model class 

class EnsembleClassifier(ClassifierI):
    
    def __init__(self, *classifiers):
        self._classifiers = classifiers
    
    # returns the classification based on majority of votes
    def classify(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)
        return mode(votes)
    # a simple measurement the degree of confidence in the classification 
    def confidence(self, features):
        votes = []
        for c in self._classifiers:
            v = c.classify(features)
            votes.append(v)

        choice_votes = votes.count(mode(votes))
        conf = choice_votes / len(votes)
        return conf


#### Loading all models using pickle

In [ ]:
# Load all classifiers from the pickled files 

# function to load models given filepath
def load_model(file_path): 
    classifier_f = open(file_path, "rb")
    classifier = pickle.load(classifier_f)
    classifier_f.close()
    return classifier


# Original Naive Bayes Classifier
ONB_Clf = load_model('pickled_algos/ONB_clf.sav')

# Multinomial Naive Bayes Classifier 
MNB_Clf = load_model('pickled_algos/MNB_clf.sav')


# Bernoulli  Naive Bayes Classifier 
BNB_Clf = load_model('pickled_algos/BNB_clf.sav')

# Logistic Regression Classifier 
LogReg_Clf = load_model('pickled_algos/LogReg_clf.sav')

# Stochastic Gradient Descent Classifier
SGD_Clf = load_model('pickled_algos/SGD_clf.sav')



# Initializing the ensemble classifier 
ensemble_clf = EnsembleClassifier(ONB_Clf, MNB_Clf, BNB_Clf, LogReg_Clf, SGD_Clf)

# List of only feature dictionary from the featureset list of tuples 
feature_list = [f[0] for f in testing_set]

# Looping over each to classify each review
ensemble_preds = [ensemble_clf.classify(features) for features in feature_list]

In [ ]:
f1_score(ground_truth, ensemble_preds, average = 'micro')

# Live Sentiment Analysis

Using the sentiment function we can classify individual reviews. 

In [ ]:
# Function to do classification a given review and return the label a
# and the amount of confidence in the classifications
def sentiment(text):
    feats = find_features(text)
    return ensemble_clf.classify(feats), ensemble_clf.confidence(feats)

In [ ]:
# sentiment analysis of reviews of captain marvel found on rotten tomatoes
text_a = '''Wow super I so very happy thank you filpcord'''
text_b = '''Got product on time, but didn't use it for few days, when I started it saw that water was leaking from somewhere, fan holder was broken, swing vent was broken, a complete defective piece
When I complained to Flipkart they said we cannot help please contact Orient customer care!!'''
text_c = '''It was lacking, a bit flat, and I'm honestly concerned about how she will enter
            the Marvel Cinematic Universe...it's so concerned with being a feminist film that 
            it forgets how to be a superhero movie.'''
text_d = '''The film may be about women breaking their shackles, but the lead actress feels kept 
            in check for much of the picture. Humor winds up being provided by Samuel Jackson's Nick 
            Fury, heart by Lashana Lynch's Maria Rambeau, and pathos by...well, it ain't Larson'''
text_e = '''"Everything was beautiful and nothing hurt"'''

sentiment(text_a), sentiment(text_b), sentiment(text_c), sentiment(text_d), sentiment(text_e)